In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
from datetime import date, timedelta
import datetime
import seaborn as sns
import numpy as np

import tensorflow as tf
from pandas_profiling import ProfileReport as PR
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose

import tensorflow.keras as ks
from tensorflow.keras import optimizers
import tensorflow as tf

from functools import partial
from scipy.special import erfinv

import datetime

import warnings
warnings.filterwarnings("ignore")
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

In [ ]:
#load data
air=pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
air.head()

#load test data
test=pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
test['date_time']=test['date_time'].apply(lambda time: datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S'))

In [ ]:
# check column information
print(air.info())

In [ ]:
# change date_time to datetime 
air['date_time']=air['date_time'].apply(lambda time: datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S'))
print(air.date_time[0])
print(air.iloc[-1,0])
air.describe()

In [ ]:
# check the distribution of each column in Train data
plt.rcParams['figure.figsize']=[10,12]
plt.subplot(3,1,1)
plt.plot(air.date_time, air.deg_C,label='deg_C')
#plt.xlabel('date time')
plt.ylabel('deg_C')

#check absolute/relative humidity
plt.subplot(3,1,2)
plt.plot(air.date_time,air.absolute_humidity,label='absolute humidity',color='green')
#plt.xlabel('date time')
plt.ylabel('absolute humidity')
plt.title('Absolute Humidity')

plt.subplot(3,1,3)
plt.plot(air.date_time,air.relative_humidity,label='relative humidity',color='orange')
plt.xlabel('date time')
plt.ylabel('relative humidity')
plt.title('Relative Humidity')
plt.show()

#distribution
plt.rcParams['figure.figsize']=[20,8]
plt.subplot(1,3,1)
plt.rcParams['figure.figsize']=[10,8]
sns.boxplot(air.absolute_humidity)
plt.subplot(1,3,2)
sns.boxplot(air.relative_humidity)
plt.subplot(1,3,3)
sns.boxplot(air.deg_C)
plt.show()

In [ ]:
#check 5 sensors 
plt.rcParams['figure.figsize']=[10,12]
plt.subplot(5,1,1)
plt.plot(air.date_time,air.sensor_1,label='sensor1',alpha=0.6)
plt.ylabel('sensor1')
plt.subplot(5,1,2)
plt.plot(air.date_time,air.sensor_2,label='sensor2',alpha=0.6)
plt.ylabel('sensor2')
plt.subplot(5,1,3)
plt.plot(air.date_time,air.sensor_3,label='sensor3',alpha=0.5)
plt.ylabel('sensor3')
plt.subplot(5,1,4)
plt.plot(air.date_time,air.sensor_4,label='sensor4',alpha=0.5)
plt.ylabel('sensor4')
plt.subplot(5,1,5)
plt.plot(air.date_time,air.sensor_5,label='sensor5',alpha=0.5)
plt.xlabel('date time')
plt.ylabel('sensor5')
plt.show()

#distribution
plt.rcParams['figure.figsize']=[20,8]
plt.subplot(1,2,1)
sns.distplot(air.sensor_1,label='sensor1')
sns.distplot(air.sensor_2,label='sensor2')
sns.distplot(air.sensor_3,label='sensor3')
sns.distplot(air.sensor_4,label='sensor4')
sns.distplot(air.sensor_5,label='sensor5')
plt.legend()

#boxplot
plt.subplot(1,2,2)
plt.boxplot(air[['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']])
plt.xticks(np.arange(1,6),['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5'])
plt.xlabel('sensor')
plt.show()

In [ ]:
# feature
X_cols=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
y_cols=['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']

# Check periodicity

In [ ]:
fft = tf.signal.rfft(air[y_cols[2]])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(air[y_cols[2]])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

In [ ]:
# seasonal decomposition
def seasonal(df,periods):
    r=df[['date_time']]
    for c in df.columns[1:]:
        result = seasonal_decompose(df[c],model='additive',period=periods)
        r['season_'+c]=result.seasonal
        r['season_'+c]=r['season_'+c].fillna(r['season_'+c].mean())
    return r

In [ ]:
gap=24
train_test=pd.concat([air.iloc[:,:-3],test.iloc[1:,:]])
season=seasonal(train_test, gap)#.reset_index()
train_test

In [ ]:
season.info()

In [ ]:
season['weekday']=np.sin(season.date_time.dt.weekday/7*np.pi/2)
season['hour']=np.sin(season.date_time.dt.hour/24*np.pi)
season['wh']=season.date_time.dt.hour.isin(list(range(7, 22, 1))).astype("int")
season['hum']=np.log1p(train_test["absolute_humidity"]*100)-np.log1p(train_test["relative_humidity"])
season['dayofyear']=np.sin(season.date_time.dt.day_of_year/365*np.pi)

In [ ]:
#add moving average
for f in ['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']:
    m=train_test[f].mean()
    s=train_test[f].std()
    mx=m+s
    mi=m-s
    
    season['mean'+f]=train_test[f].rolling(window=1,center=True).mean().fillna(m)
    season['max'+f]=train_test[f].rolling(window=1,center=True).max().fillna(m)
    season['min'+f]=train_test[f].rolling(window=1,center=True).min().fillna(m)
    
season.columns

In [ ]:
#feature
X_cols_scaled=['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5',
              'season_deg_C','season_relative_humidity','season_absolute_humidity', 'season_sensor_1', 'season_sensor_2','season_sensor_3', 'season_sensor_4', 'season_sensor_5',
              'meansensor_1', 'maxsensor_1','minsensor_1', 'meansensor_2', 'maxsensor_2', 'minsensor_2',
               'meansensor_3', 'maxsensor_3', 'minsensor_3', 'meansensor_4','maxsensor_4', 'minsensor_4', 'meansensor_5', 'maxsensor_5','minsensor_5']
X_cols_scaled_time=['date_time','deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5',
              'season_deg_C','season_relative_humidity','season_absolute_humidity', 'season_sensor_1', 'season_sensor_2','season_sensor_3', 'season_sensor_4', 'season_sensor_5',
              'meansensor_1', 'maxsensor_1','minsensor_1', 'meansensor_2', 'maxsensor_2', 'minsensor_2',
               'meansensor_3', 'maxsensor_3', 'minsensor_3', 'meansensor_4','maxsensor_4', 'minsensor_4', 'meansensor_5', 'maxsensor_5','minsensor_5']
X_cols_season=['date_time','deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5',
              'season_deg_C','season_relative_humidity','season_absolute_humidity', 'season_sensor_1', 'season_sensor_2','season_sensor_3', 'season_sensor_4', 'season_sensor_5',
               'meansensor_1', 'maxsensor_1','minsensor_1', 'meansensor_2', 'maxsensor_2', 'minsensor_2',
               'meansensor_3', 'maxsensor_3', 'minsensor_3', 'meansensor_4','maxsensor_4', 'minsensor_4', 'meansensor_5', 'maxsensor_5','minsensor_5',
              'weekday','hour','wh','hum','dayofyear']
X_cols_no_time=['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5',
              'season_deg_C','season_relative_humidity','season_absolute_humidity', 'season_sensor_1', 'season_sensor_2','season_sensor_3', 'season_sensor_4', 'season_sensor_5',
                'meansensor_1', 'maxsensor_1','minsensor_1', 'meansensor_2', 'maxsensor_2', 'minsensor_2',
               'meansensor_3', 'maxsensor_3', 'minsensor_3', 'meansensor_4','maxsensor_4', 'minsensor_4', 'meansensor_5', 'maxsensor_5','minsensor_5',
                'weekday','hour','wh','hum','dayofyear']
               # 'meandeg_C', 'maxdeg_C', 'mindeg_C','meanrelative_humidity', 'maxrelative_humidity', 'minrelative_humidity','meanabsolute_humidity', 'maxabsolute_humidity','minabsolute_humidity',
all_cols=['date_time','deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5',
          'season_deg_C','season_relative_humidity','season_absolute_humidity', 'season_sensor_1', 'season_sensor_2','season_sensor_3', 'season_sensor_4', 'season_sensor_5',
          'meansensor_1', 'maxsensor_1','minsensor_1', 'meansensor_2', 'maxsensor_2', 'minsensor_2',
               'meansensor_3', 'maxsensor_3', 'minsensor_3', 'meansensor_4','maxsensor_4', 'minsensor_4', 'meansensor_5', 'maxsensor_5','minsensor_5',
          'weekday','hour','wh','hum','dayofyear',
          'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
print(air.shape,season.shape)

# Make Time-series data

In [ ]:
def make_timeseries_data(data,nsteps,X_cols,y_cols=False):
    X=[]
    y=[]
    for t in range(len(data)-nsteps+1):
        d=t+nsteps #len(data)
        X.append(data[X_cols].iloc[t:d,1:].values)
        if t==0:
            print('start X time:')
            print(data.iloc[t:d,0])
        if t==len(data)-nsteps:
                print('final X time:')
                print(data.iloc[t:d,0])
    
    if y_cols:
        for t in range(len(data)-nsteps+1):
            d=t+nsteps-1
            if t==0:
                print('start target time:')
                print(data.iloc[d,0])
            if t==len(data)-nsteps:
                print('final target time:')
                print(data.iloc[d,0])
            y.append(data[y_cols].iloc[d,:].values)
    if y_cols:
        return np.array(X),np.array(y)
    return np.array(X)

In [ ]:
window=20

#scale - StandardScaler
scaler=StandardScaler()
scaler_target=StandardScaler()

#feature 2010-03-10 18:00:00 ~ 2011-04-04 14:00:00, 9357 rows
df_scaled=scaler.fit_transform(air_season_feature[X_cols_scaled]) # scale to all data including train and test
df_scaled=pd.DataFrame(df_scaled)
df_feature=pd.concat([air_season_feature[['date_time']],df_scaled],axis=1)
df_feature.columns=X_cols_scaled_time
df_feature=pd.concat([df_feature,season.reset_index()[['weekday','hour','wh','hum','dayofyear']]],axis=1)
df_feature

#air - 2010-03-10 18:00:00 ~ 2011-01-01 00:00:00, 7111 rows
#target - 2010-03-10 18:00:00 ~ 2011-01-01 00:00:00, 7111 rows
target_scaled=scaler_target.fit_transform(air[y_cols]) # scaled to only train data
target_scaled=pd.DataFrame(target_scaled)
target_scaled=pd.concat([air[['date_time']].iloc[:].reset_index(drop=True),target_scaled],axis=1)#.reset_index()
target_scaled.columns=['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']

#feature - 2010-03-10 18:00:00 ~ 2010-12-31 04:00:00
#target - 2010-03-11 14:00:00 ~ 2011-01-01 00:00:00	
#merged
train_scaled=pd.concat([df_feature.iloc[:len(target_scaled),:],target_scaled.iloc[:,1:]],axis=1) # including feature and target
train_scaled#.columns

# Train data
X,y=make_timeseries_data(train_scaled,window,X_cols_no_time,y_cols)

In [ ]:
#check dimension
print('total shape: ',df_scaled.shape)
print('train shape: ',train_scaled.shape)
print('time-series shape: ',X.shape)
print('test feature shape: ',len(test)-1)
print('Number of features: ',len(X_cols_season)-1)
df_feature

In [ ]:
# Test data
# 2010-12-31 04:00:00 ~ 2011-04-04 13:00:00
# last data 2011-04-04 14:00:00
test_scaled=df_feature.iloc[-len(test)-window+1:,:] # including only feature

X_test=make_timeseries_data(test_scaled,window,X_cols_no_time)

X_test.shape

In [ ]:
#split data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_valid.shape)

# Build model

In [ ]:
#window=20 now - 0.27171
#best 0.36449 - 0.26686
lstm_model=keras.models.Sequential([
    keras.layers.LSTM(128,kernel_initializer='LecunUniform',activation = "tanh",return_sequences= True),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(32,kernel_initializer='LecunUniform',activation = "sigmoid",return_sequences=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(3,activation =keras.layers.PReLU(),kernel_initializer='LecunUniform')
])

In [ ]:
#!pip install keras-tcn

In [ ]:
# TCN
# nb_stacks=3,use_batch_norm=True,dropout_rate=0.2
#from tcn import TCN, tcn_full_summary
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.models import Sequential

#tcn_layer=TCN(nb_filters=32,nb_stacks=2,kernel_size=8,use_batch_norm=True,input_shape=(X_train.shape[1],X_train.shape[2]))
# The receptive field tells you how far the model can see in terms of timesteps.
#print('Receptive field size =', tcn_layer.receptive_field)

#tcn_model = Sequential([
#    tcn_layer,
#    Dense(3)
#])

In [ ]:
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
xgb_model1 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_model2 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)           
xgb_model3 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)           

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search1 = RandomizedSearchCV(xgb_model1, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)
search2 = RandomizedSearchCV(xgb_model2, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)
search3 = RandomizedSearchCV(xgb_model3, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search1.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[0]])
search2.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[1]])
search3.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[2]])

report_best_scores(search1.cv_results_, 1)
report_best_scores(search2.cv_results_, 1)
report_best_scores(search3.cv_results_, 1)

In [ ]:
xgb_model1 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42,
                             colsample_bytree=0.7314607499597442,gamma=0.36870260304690955,learning_rate=0.0846851635427874,
                              max_depth=4,n_estimators=116,subsample=0.6845792027986178)
xgb_model2 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42,
                             colsample_bytree=0.7190535488770003,gamma=0.01841093381799569,learning_rate=0.07015563564018995,
                              max_depth=2,n_estimators=147,subsample=0.6301436241409849)           
xgb_model3 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42,
                             colsample_bytree=0.9040922615763338,gamma=0.2252496259847715,learning_rate=0.033979488347959955,
                              max_depth=2,n_estimators=113,subsample=0.9233589392465844)    

xgb_model1.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[0]])
xgb_model2.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[1]])
xgb_model3.fit(air_season_feature[X_cols_no_time].iloc[:len(air)],air[y_cols[2]])

y_pred1=xgb_model1.predict(air_season_feature[X_cols_no_time].iloc[-len(X_test):])
y_pred2=xgb_model2.predict(air_season_feature[X_cols_no_time].iloc[-len(X_test):])
y_pred3=xgb_model3.predict(air_season_feature[X_cols_no_time].iloc[-len(X_test):])
print(y_pred1.shape)

In [ ]:
xgb1=xgb_model1.predict(air_season_feature[X_cols_no_time].iloc[:len(air)])
xgb2=xgb_model2.predict(air_season_feature[X_cols_no_time].iloc[:len(air)])
xgb3=xgb_model3.predict(air_season_feature[X_cols_no_time].iloc[:len(air)])

xgb=np.array([xgb1,xgb2,xgb3]).transpose()

# Compile model

In [ ]:
MAX_EPOCHS = 400

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=5,
                                                    mode='min')
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)
def piecewise_constant_fn(epoch):
    if epoch<5:
        return 0.01
    elif epoch<15:
        return 0.005
    elif epoch<40:
        return 0.001
    else:
        return 0.0005

lr_scheduler= keras.callbacks.LearningRateScheduler(exponential_decay_fn)                             
checkpoint= ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
optimizer=keras.optimizers.SGD(clipvalue=100,momentum=0.85,nesterov=True)

lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

history=lstm_model.fit(X_train,y_train,epochs=MAX_EPOCHS,#batch_size=32,
                      validation_data=(X_valid,y_valid),
                      callbacks=[early_stopping,lr_scheduler,checkpoint])

In [ ]:
lstm_model.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    #plt.axis([1, 20, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
lstm_pred=lstm_model.predict(X_test)

lstm_pred=scaler_target.inverse_transform(lstm_pred)

submission=sample[['date_time']]
submission['target_carbon_monoxide']=lstm_pred[:,0]
submission['target_benzene']=lstm_pred[:,1]
submission['target_nitrogen_oxides']=lstm_pred[:,2]

lstm_pred.shape

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
print('LSTM is better than Xgboost')